In [2]:
import pandas as pd
import numpy as np
import re as re
import xgboost as xgb
# Going to use these 4 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold

train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
PassengerId = test['PassengerId']

In [3]:

#################################################################################################### 
#                                       PRE-PROCESSING                                             #
#################################################################################################### 

# This part essentially ripped from Sina's work as I'm too lazy

full_data = [train, test]
# Check distribution of PCLASS and number survived
print(train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())
# Check distribution of Sexes and number survived
print(train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean())
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)
# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
# Encoding our features
for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']                           = 4

# Feature Selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
#drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp','Parch', 'FamilySize']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363
      Sex  Survived
0  female  0.742038
1    male  0.188908


/Users/zhesong/my-virtualenv/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
#################################################################################################### 
#                                         STACKING                                                 #
#################################################################################################### 

ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Write some Python helper functions that collects a lot of the SKlearn methods under one roof. 
# Totally ripped from Faron's Stacking starter ;)
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)


def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)
    
# Assign the parameters for each of our 4 base models
rf_params = {
    'n_jobs': -1,
    'n_estimators': 575,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 3 
}
et_params = {
    'n_jobs': -1,
    'n_estimators':575,
    #'max_features': 0.5,
    'max_depth': 5,
    'min_samples_leaf': 3,
    'verbose': 3
}
ada_params = {
    'n_estimators': 575,
    'learning_rate' : 0.95
}

gb_params = {
    'n_estimators': 575,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 3,
    'verbose': 3
}
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }


# Create 4 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis=1)
x_train = train.values
x_test = test.values

# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test)
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test)
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test)
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test)
svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test)

x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)
print("{},{}".format(x_train.shape, x_test.shape))

# Finally, we use an Xgboost classifier and feed it our oof train and test values as new features
gbm = xgb.XGBClassifier(learning_rate = 0.95,
 n_estimators= 5000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=1,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, y_train)
predictions = gbm.predict(x_test)

building tree 1 of 575
building tree 2 of 575building tree 3 of 575building tree 4 of 575
building tree 5 of 575

building tree 6 of 575
building tree 7 of 575

building tree 8 of 575
building tree 9 of 575
building tree 10 of 575
building tree 11 of 575
building tree 12 of 575building tree 13 of 575
building tree 14 of 575

building tree 15 of 575
building tree 16 of 575
building tree 17 of 575
building tree 18 of 575
building tree 19 of 575building tree 20 of 575building tree 21 of 575


building tree 22 of 575building tree 23 of 575

building tree 24 of 575
building tree 25 of 575building tree 26 of 575

building tree 27 of 575building tree 28 of 575

building tree 29 of 575building tree 30 of 575

building tree 31 of 575
building tree 32 of 575building tree 33 of 575

building tree 34 of 575building tree 35 of 575

building tree 36 of 575
building tree 37 of 575
building tree 38 of 575
building tree 39 of 575
building tree 40 of 575building tree 41 of 575

building tree 42 of 575bu

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.2s


building tree 174 of 575
building tree 175 of 575
building tree 176 of 575
building tree 177 of 575building tree 178 of 575
building tree 179 of 575building tree 180 of 575


building tree 181 of 575
building tree 182 of 575
building tree 183 of 575
building tree 184 of 575building tree 185 of 575
building tree 186 of 575

building tree 187 of 575
building tree 188 of 575
building tree 189 of 575
building tree 190 of 575
building tree 191 of 575building tree 192 of 575

building tree 193 of 575
building tree 194 of 575building tree 195 of 575

building tree 196 of 575
building tree 197 of 575
building tree 198 of 575
building tree 199 of 575
building tree 200 of 575
building tree 201 of 575
building tree 202 of 575building tree 203 of 575
building tree 204 of 575

building tree 205 of 575
building tree 206 of 575
building tree 207 of 575
building tree 208 of 575
building tree 209 of 575building tree 210 of 575

building tree 211 of 575
building tree 212 of 575
building tree 213 of 575


[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:    0.3s


building tree 346 of 575
building tree 347 of 575building tree 348 of 575

building tree 349 of 575
building tree 350 of 575
building tree 351 of 575
building tree 352 of 575building tree 353 of 575

building tree 354 of 575
building tree 355 of 575
building tree 356 of 575
building tree 357 of 575
building tree 358 of 575
building tree 359 of 575
building tree 360 of 575building tree 361 of 575

building tree 362 of 575building tree 363 of 575
building tree 364 of 575

building tree 365 of 575
building tree 366 of 575
building tree 367 of 575
building tree 368 of 575
building tree 369 of 575building tree 370 of 575

building tree 371 of 575
building tree 372 of 575
building tree 373 of 575building tree 374 of 575

building tree 375 of 575building tree 376 of 575

building tree 377 of 575
building tree 378 of 575building tree 379 of 575

building tree 380 of 575
building tree 381 of 575
building tree 382 of 575
building tree 383 of 575building tree 384 of 575

building tree 385 of 575


...

       264           0.4742            0.55s
       265           0.4740            0.55s
       266           0.4736            0.54s
       267           0.4735            0.54s
       268           0.4734            0.54s
       269           0.4732            0.54s
       270           0.4732            0.54s
       271           0.4731            0.54s
       272           0.4728            0.54s
       273           0.4727            0.54s
       274           0.4726            0.54s
       275           0.4724            0.54s
       276           0.4722            0.53s
       277           0.4721            0.53s
       278           0.4720            0.53s
       279           0.4719            0.53s
       280           0.4718            0.53s
       281           0.4716            0.53s
       282           0.4714            0.53s
       283           0.4713            0.53s
       284           0.4711            0.52s
       285           0.4710            0.52s
       286

       469           0.4574            0.20s
       470           0.4573            0.19s
       471           0.4573            0.19s
       472           0.4573            0.19s
       473           0.4573            0.19s
       474           0.4572            0.19s
       475           0.4572            0.19s
       476           0.4572            0.18s
       477           0.4572            0.18s
       478           0.4572            0.18s
       479           0.4572            0.18s
       480           0.4572            0.18s
       481           0.4571            0.17s
       482           0.4571            0.17s
       483           0.4571            0.17s
       484           0.4571            0.17s
       485           0.4571            0.17s
       486           0.4571            0.17s
       487           0.4570            0.16s
       488           0.4570            0.16s
       489           0.4570            0.16s
       490           0.4570            0.16s
       491

       114           0.5538            0.82s
       115           0.5524            0.82s
       116           0.5522            0.81s
       117           0.5519            0.81s
       118           0.5516            0.81s
       119           0.5499            0.81s
       120           0.5496            0.81s
       121           0.5488            0.81s
       122           0.5480            0.80s
       123           0.5467            0.80s
       124           0.5459            0.80s
       125           0.5453            0.80s
       126           0.5448            0.79s
       127           0.5443            0.79s
       128           0.5432            0.79s
       129           0.5423            0.79s
       130           0.5418            0.79s
       131           0.5413            0.78s
       132           0.5406            0.78s
       133           0.5398            0.78s
       134           0.5394            0.78s
       135           0.5390            0.77s
       136

       338           0.4803            0.42s
       339           0.4803            0.42s
       340           0.4802            0.42s
       341           0.4802            0.42s
       342           0.4801            0.42s
       343           0.4799            0.42s
       344           0.4799            0.41s
       345           0.4799            0.41s
       346           0.4799            0.41s
       347           0.4798            0.41s
       348           0.4798            0.40s
       349           0.4798            0.40s
       350           0.4797            0.40s
       351           0.4797            0.40s
       352           0.4796            0.40s
       353           0.4794            0.40s
       354           0.4794            0.39s
       355           0.4793            0.39s
       356           0.4792            0.39s
       357           0.4788            0.39s
       358           0.4787            0.39s
       359           0.4787            0.38s
       360

       560           0.4648            0.03s
       561           0.4647            0.03s
       562           0.4647            0.02s
       563           0.4647            0.02s
       564           0.4647            0.02s
       565           0.4647            0.02s
       566           0.4647            0.02s
       567           0.4647            0.01s
       568           0.4647            0.01s
       569           0.4646            0.01s
       570           0.4646            0.01s
       571           0.4646            0.01s
       572           0.4646            0.01s
       573           0.4646            0.00s
       574           0.4645            0.00s
       575           0.4645            0.00s
(891, 5),(418, 5)


In [5]:
#################################################################################################### 
#                           PRODUCING SUBMISSION FILE                                              #
#################################################################################################### 
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })
StackingSubmission.to_csv("stacking_submission_v2.csv", index=False)